# 前処理大全

# 第一章 前処理とは

### データ分析の３つの前処理
- 表やグラフ作成用の前処理
- 教師なし学習へ入力するための前処理
- 教師あり学習へ入力するための前処理

### 前処理の流れ  
#### 1 データ構造を対象とした処理:大きなデータを扱うので、SQLが有利。  
- 対象データの抽出や結合、集約など  
 
#### 2 データ内容を対象とした処理：値の修正などのため、python有利。  
- データ内容の変更や、欠損値の補完など  
→ 集計やグラフの描写に使用  
- 機械学習モデルのために変換  
→ 教師なし学習用データ  
    
#### 3 データ構造を対象とした処理:機械学習用のライブラリの多いpythonを使用。  
- 学習、テストデータの分割  
→ 教師あり学習用データ  

## 第二章 抽出
データサイズを小さくするメリットは多いので、必要なデータのみを抽出するようにする。

### 注意事項
- iloc、ixなど値で指定する方法は不推奨、locや配列に文字列を使用する事を推奨。
- indexが無い場合、条件指定時に全探索する事になる。indexがあれば、必要範囲の指定となるので処理速度が向上。
- 条件指定抽出は、query関数が一番視認性が良いので推奨。
- データ量を指定の数に減らせるsample関数は重要であるが、指定する項目の選択によってはデータ割合に支障をきたすので注意が必要。

# 第三章 集約

集約方法は大きく分けると２つの方法がある  
- groupby    : 条件表現が豊富、但し記述量が多い。
- Window関数  : SQLが簡単に書けてオススメ！！ グループ毎に並び替えて順位づけする事に長けている。

**注意事項**  
- 集約処理とwindow関数の実行を同時にはできない。（SQLはできる)


### aggが便利
- agg({"指定先1":処理内容1,"指定先2":処理内容})と記述するだけで、複数の処理を行ってくれる。 
- １つの処理の時には、記述量が増えるので使わない。

# 第四章 結合

### マスターテーブルとレコードテーブル
- マスターテーブルは、顧客の情報などの共通情報を保管しており、顧客IDを持っている。
- レコードテーブルは、商品情報や店舗情報など様々な種類がある。

レコードテーブルにも顧客IDを所持するものもあり、マスターテーブルと結合することで、顧客情報の紐付けができる。

### 注意事項  
結合前になるべく不要なもの除くために抽出しておく。  

### 過去データの取扱　　
- 結合対象の期間を絞る。
- 結合した過去データに集約関数を利用する。 window関数が最適、但しpythonは実装されてないのでrolling関数を利用する。  
  rolling: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html

# 第五章 分割  
SQLでの作業は全く効率的でないためしない事。

## 時系列データにおける処理
- 単純な交差検証は有効ではない。未来のデータが混じっている可能性がある。
- 上記の対策として、期間をスライドさせたり、期間を追加していく検証方法が有効。  
- 期間を追加して行く場合は、追加する時期によりデータ量が変わってくるので、データ量の増加に伴う精度の関係も把握する必要がある。

**注意事項**
- pythonには時系列データを簡単に扱えるライブラリは無い。

In [29]:
import pandas as pd
import numpy as np

df = pd.read_csv("awesomebook/data/month_mst.csv")
df["year_month"] = df["year_num"].astype(str) + "-" + df["month_num"].astype(str)
df["year_month"] = df["year_month"].sort_values(ascending=True)
df.sort_values(by="year_month")


train_window_start = 1
train_window_end   = 24
horizon = 12
skip    = 12

while True:
    test_window_end = train_window_end + horizon
    train = df[train_window_start:train_window_end]
    test  = df[(train_window_end + 1):test_window_end]
    if test_window_end >= len(df.index):
        break

    train_window_start += skip
    train_window_end += skip

# 第六章 生成

## 不均等データの調整

障害でないデータに対して、障害データが極端に少ないケースを不均等と言う。
不均等なデータは機械学習の予測精度が下がる要因となる可能性が多い。

## 不均等なデータへの対策
- 機械学習のモデル制作時に重みを与える。
- データを操作して不均等な状態を解除する。  
  -少ないデータを増やす : オーバーサンプリング  
  -多いデータを減らす   : アンダーサンプリング  
  -この両方を行う方法
  
## アンダーサンプリング
多い方のデータを減らすだけなので、簡単に実装出来ます。。  
但し、データを減らす事は勿体無いので、基本的にはオーバーサンプリングを行います。  
オーバーサンプリングで過学習を起こしそうな場合に、両方行うようにする事が望ましい。

## オーバーサンプリング  
データを増やす時に使う、増やす手法としてSMOTEが使いやすい。  
SMOTEは、障害データとその選択候補となるデータの中間に新たなデータを作る手法。

In [ ]:
from 